In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas
import gensim

import matplotlib.pyplot as plt
import matplotlib

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout

matplotlib.style.use('ggplot')

Using Theano backend.


## 1. Read data

In [3]:
data_file = "../data/bdata.log.txt"

data = pandas.read_csv(data_file, sep = ',')

In [4]:
data.head()

species       mhc  peptide_length   cv   sequence inequality      meas
0     cow  BoLA-HD6               9  TBD  ALFYKDGKL          =  1.000000
1     cow  BoLA-HD6               9  TBD  ALYEKKLAL          =  1.000000
2     cow  BoLA-HD6               9  TBD  AMKDRFQPL          =  0.860543
3     cow  BoLA-HD6               9  TBD  AQRELFFTL          =  1.000000
4     cow  BoLA-HD6               9  TBD  FMKVKFEAL          =  0.957914

## 2. To one hot

In [5]:
selected_mhc = 'HLA-A*03:01'

indexes = data['mhc'][data['mhc'] == selected_mhc].index
indexes = data.iloc[indexes]['mhc'][data['peptide_length'] == 9].index
#indexes = data.index
selected_X = data['sequence'][indexes]
selected_y = data['meas'][indexes]

letters_X = selected_X.apply(list)

selected_X = pandas.DataFrame(list(letters_X))
selected_X = pandas.get_dummies(selected_X)

XX = selected_X[range(0, 20)].as_matrix() + selected_X[range(20, 40)].as_matrix() + selected_X[range(40, 60)].as_matrix() + selected_X[range(60, 80)].as_matrix() + selected_X[range(80, 100)].as_matrix() + selected_X[range(100, 120)].as_matrix() + selected_X[range(120, 140)].as_matrix() + selected_X[range(140, 160)].as_matrix() + selected_X[range(160, 180)].as_matrix()
selected_X = pandas.concat([selected_X, pandas.DataFrame(XX)], axis=1)

selected_X.shape

(5231, 200)

## 3. ml stuff

In [6]:
#normalize output
# m = selected_y.mean()
# d = selected_y.max() - selected_y.min()
# selected_y = (selected_y - m) / d

# train/test split
random_number = 52
X_train, X_test, y_train, y_test = train_test_split(selected_X, selected_y,
                                                    test_size = 0.33, random_state = random_number)

In [7]:
def ridge_regression(X, y, alpha):
    #Fit the model
    ridgereg = Ridge(alpha=alpha,normalize=True)
    ridgereg.fit(X, y)
    y_pred = ridgereg.predict(X)
    
    #Return the result in pre-defined format
    rss = sum((y_pred - y)**2)
    return rss, ridgereg.intercept_, ridgereg

In [8]:
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2]

n = 6

rss = [0] * n
intercept = [0] * n
predictors = [0] * n

for i in range(n):
    rss[i], intercept[i], predictors[i] = ridge_regression(X_train, y_train, alpha_ridge[i])

In [9]:
# predict test data

test_rss = [0] * n
for i in range(n):
    m_pred = predictors[i]
    test_pred = m_pred.predict(X_test)
    test_rss[i] = sum((test_pred - y_test)**2)
    
    print '-' * 10
    print "alpha = ", alpha_ridge[i]
    print "train rss = ", rss[i]
    print "test rss = ", test_rss[i]
    
m_pred = predictors[5]
test_pred = m_pred.predict(X_test)

----------
alpha =  1e-15
train rss =  88.9680144288
test rss =  48.7164963496
----------
alpha =  1e-10
train rss =  88.9680144288
test rss =  48.7164963494
----------
alpha =  1e-08
train rss =  88.9680144288
test rss =  48.7164963378
----------
alpha =  0.0001
train rss =  88.9680151731
test rss =  48.7163790007
----------
alpha =  0.001
train rss =  88.968088723
test rss =  48.7153589064
----------
alpha =  0.01
train rss =  88.9753067544
test rss =  48.7086515093


In [10]:
test_pred = pandas.DataFrame(test_pred)

In [11]:
y_test = pandas.DataFrame(y_test)

In [12]:
test_pred['1'] = np.array(y_test)

In [13]:
test_pred.corr()

0         1
0  1.000000  0.757778
1  0.757778  1.000000

## NN

In [14]:
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()

y_train = y_train.as_matrix().flatten()
y_test = y_test.as_matrix().flatten()

In [15]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(1))

model.compile(loss='mse', optimizer='rmsprop')

In [16]:
model.fit(X_train, y_train, batch_size=32, nb_epoch=500, verbose=2, validation_split=0.15)

Train on 2978 samples, validate on 526 samples
Epoch 1/500
0s - loss: 0.0594 - val_loss: 0.0400
Epoch 2/500
0s - loss: 0.0381 - val_loss: 0.0333
Epoch 3/500
0s - loss: 0.0291 - val_loss: 0.0271
Epoch 4/500
0s - loss: 0.0230 - val_loss: 0.0252
Epoch 5/500
0s - loss: 0.0189 - val_loss: 0.0270
Epoch 6/500
0s - loss: 0.0168 - val_loss: 0.0248
Epoch 7/500
0s - loss: 0.0138 - val_loss: 0.0290
Epoch 8/500
0s - loss: 0.0123 - val_loss: 0.0237
Epoch 9/500
0s - loss: 0.0115 - val_loss: 0.0247
Epoch 10/500
0s - loss: 0.0098 - val_loss: 0.0244
Epoch 11/500
0s - loss: 0.0094 - val_loss: 0.0254
Epoch 12/500
0s - loss: 0.0083 - val_loss: 0.0251
Epoch 13/500
0s - loss: 0.0080 - val_loss: 0.0236
Epoch 14/500
0s - loss: 0.0071 - val_loss: 0.0241
Epoch 15/500
0s - loss: 0.0067 - val_loss: 0.0239
Epoch 16/500
0s - loss: 0.0063 - val_loss: 0.0271
Epoch 17/500
0s - loss: 0.0063 - val_loss: 0.0246
Epoch 18/500
0s - loss: 0.0061 - val_loss: 0.0235
Epoch 19/500
0s - loss: 0.0055 - val_loss: 0.0240
Epoch 20/500

In [17]:
preds = model.predict(X_test, verbose=0).flatten()

In [18]:
sum((preds - y_test)**2)

41.482796378169191